# TQuant Lab SuperTrend Strategy - Sinopac Shioaji

## Overview of the SuperTrend Strategy
The SuperTrend Strategy combines the SuperTrend Indicator and the ADX Indicator. The SuperTrend Indicator consists of upper and lower bands. When the stock price breaks above the upper band, it signals a breakout, indicating a potential upward trend. Conversely, when the price drops below the lower band, it indicates a breach of support and a potential downward trend.

The ADX (Average Directional Index), ranging from 0 to 100, measures trend strength. Higher values signify stronger trends, helping confirm bullish or bearish conditions出。

詳細的策略說明與 TQuant Lab 的回測流程請參考：[TQuant Lab 超級趨勢策略，低買高賣賺取波段價差](https://www.tejwin.com/insight/tquant-lab-%E8%B6%85%E7%B4%9A%E8%B6%A8%E5%8B%A2%E7%AD%96%E7%95%A5/)

## Identifying Daily Trading Targets

In [1]:
import os
import numpy as np
import pandas as pd

# tej_key
tej_key = 'your key'
api_base = 'https://api.tej.com.tw'

os.environ['TEJAPI_KEY'] = tej_key 
os.environ['TEJAPI_BASE'] = api_base

In [2]:
from zipline.utils.calendar_utils import get_calendar
from zipline.sources.TEJ_Api_Data import get_universe

cal = get_calendar('TEJ').all_sessions
last_date = cal[-2]  # last trading date

pool = get_universe(start = last_date, 
                    end = last_date,
                    mkt_bd_e = 'TSE',  # Listed stock in Taiwan
                    stktp_e = 'Common Stock', 
                    main_ind_e = 'M2300 Electronics'  # Electronics Industry
                    )

Currently used TEJ API key call quota 230/100000 (0.23%)
Currently used TEJ API key data quota 787917/10000000 (7.88%)


In [3]:
len(pool)

407

In [4]:
import TejToolAPI

mktcap_data = TejToolAPI.get_history_data(start = last_date,
                                          end = last_date,
                                          ticker = pool,
                                          columns = ['Market_Cap_Dollars']
                                         )

mktcap_data

Currently used TEJ API key call quota 260/100000 (0.26%)
Currently used TEJ API key data quota 900607/10000000 (9.01%)


,coid,mdate,Market_Cap_Dollars
0,1471,2024-10-25,1.962910e+09
1,1582,2024-10-25,1.514820e+10
2,2059,2024-10-25,1.195978e+11
3,2301,2024-10-25,2.441140e+11
4,2302,2024-10-25,3.201330e+09
...,...,...,...
402,8215,2024-10-25,1.032572e+10
403,8249,2024-10-25,9.068506e+09
404,8261,2024-10-25,9.226725e+09
405,8271,2024-10-25,6.603811e+09


In [5]:
tickers = mktcap_data.nlargest(100, 'Market_Cap_Dollars')['coid'].tolist()
tickers

['2330',
 '2317',
 '2454',
 '2382',
 '2308',
 '2412',
 '3711',
 '2303',
 '2357',
 '3045',
 '6669',
 '2345',
 '3231',
 '4904',
 '2327',
 '3008',
 '3034',
 '2395',
 '4938',
 '3017',
 '3037',
 '2379',
 '2301',
 '3653',
 '3533',
 '6409',
 '2376',
 '2360',
 '3443',
 '2356',
 '2474',
 '2449',
 '2324',
 '2377',
 '2383',
 '2408',
 '2409',
 '3481',
 '3702',
 '3036',
 '2353',
 '5269',
 '2385',
 '2347',
 '2059',
 '6526',
 '2354',
 '3044',
 '6239',
 '6176',
 '2368',
 '6789',
 '2344',
 '8046',
 '6770',
 '2313',
 '2352',
 '3005',
 '3035',
 '2388',
 '2404',
 '3023',
 '6285',
 '5434',
 '6805',
 '3706',
 '6139',
 '3189',
 '3406',
 '2492',
 '6412',
 '6531',
 '2337',
 '3532',
 '2458',
 '6515',
 '4915',
 '8070',
 '6414',
 '2451',
 '4919',
 '3042',
 '2312',
 '6257',
 '2498',
 '2363',
 '2362',
 '3413',
 '3583',
 '2393',
 '6442',
 '6214',
 '8112',
 '8210',
 '3376',
 '3714',
 '3596',
 '5388',
 '3030',
 '6691']

In [6]:
from datetime import timedelta

start = (last_date - timedelta(days=90)).strftime('%Y-%m-%d')  # We want 3 months (90 days) data
end = last_date.strftime('%Y-%m-%d')

os.environ['mdate'] = start + ' ' + end
os.environ['ticker'] = ' '.join(tickers)

!zipline ingest -b tquant

Merging daily equity files:
Currently used TEJ API key call quota 266/100000 (0.27%)
Currently used TEJ API key data quota 914122/10000000 (9.14%)


[2024-10-28 03:45:19.943089] INFO: zipline.data.bundles.core: Ingesting tquant.
[2024-10-28 03:45:22.401729] INFO: zipline.data.bundles.core: Ingest tquant successfully.


In [7]:
from zipline.pipeline import CustomFactor
from zipline.pipeline.data import TWEquityPricing

class Supertrend(CustomFactor):
    inputs = [
        TWEquityPricing.high,
        TWEquityPricing.low,
        TWEquityPricing.close,
    ]
    window_length = 50  # Use a 50-day window length
    outputs = ['atr', 'basic_upperband', 'basic_lowerband', 'final_upperband', 'final_lowerband']

    def compute(self, today, assets, out, highs, lows, closes):
        # Calculate TR (True Range)
        high_low = highs[1:] - lows[1:]
        high_close = abs(highs[1:] - closes[:-1])
        low_close = abs(lows[1:] - closes[:-1])
        tr = np.maximum.reduce([high_low, high_close, low_close])

        # Calculate ATR (Average True Range)
        atr = np.mean(tr, axis=0)
        out.atr = atr

        # Calculate basic upperband & lowerband
        hl2 = (highs + lows) / 2
        basic_upperband = hl2 + 4 * atr
        basic_lowerband = hl2 - 4 * atr

        # Initialize final upperband & lowerband
        final_upperband = np.zeros_like(basic_upperband)
        final_lowerband = np.zeros_like(basic_lowerband)

        # Calculate fianl upperband & lowerband
        for i in range(1, len(closes)):
            final_upperband[i] = np.where(
                (basic_upperband[i] < final_upperband[i-1]) | (closes[i-1] > final_upperband[i-1]),
                basic_upperband[i],
                final_upperband[i-1]
            )
            
            final_lowerband[i] = np.where(
                (basic_lowerband[i] > final_lowerband[i-1]) | (closes[i-1] < final_lowerband[i-1]),
                basic_lowerband[i],
                final_lowerband[i-1]
            )
            
        out.basic_upperband = basic_upperband[-1]
        out.basic_lowerband = basic_lowerband[-1]
        out.final_upperband = final_upperband[-1]
        out.final_lowerband = final_lowerband[-1]

In [8]:
class ADX(CustomFactor):
    inputs = [TWEquityPricing.high, TWEquityPricing.low, TWEquityPricing.close]
    window_length = 14  # The ADX is typically used with a default period of 14 days
    outputs = ['adx']
    
    def compute(self, today, assets, out, highs, lows, closes):
        # Calculate TR (True Range)
        high_low = highs[1:] - lows[1:]
        high_close = abs(highs[1:] - closes[:-1])
        low_close = abs(lows[1:] - closes[:-1])
        tr = np.maximum.reduce([high_low, high_close, low_close])

        # Calculate ATR (Average True Range)
        atr = np.mean(tr, axis=0)
        
        # Calculate +DM, -DM (Directional Movement)
        up_move = highs[1:] - lows[:-1]
        down_move = lows[:-1] - lows[1:]
        plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0)
        minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0)
        
        # Calaulate rolling mean of +DM & -DM (smoothing)
        plus_dm_smooth = np.mean(plus_dm, axis=0)
        minus_dm_smooth = np.mean(minus_dm, axis=0)
        
        # Calculate +DI, -DI (Directional Indicator)
        di_plus = 100 * np.divide(plus_dm_smooth, atr)
        di_minus = 100 * np.divide(minus_dm_smooth, atr)
        
        # Calculate DX
        dx = 100 * np.divide(np.abs(di_plus - di_minus), np.abs(di_plus + di_minus))
        
        # Calculate ADX (rolling mean of DX)
        dx_series = pd.DataFrame(dx)
        adx = dx_series.rolling(window = 50, min_periods = 1).mean().values.flatten()

        out.adx = adx

In [9]:
from zipline.pipeline import Pipeline
from zipline.TQresearch.tej_pipeline import run_pipeline
from zipline.pipeline.filters import StaticAssets

def make_pipeline():

    close = TWEquityPricing.close.latest
    supertrend = Supertrend()
    adx = ADX()
    
    return Pipeline(
        columns={
            'close': close,
            'final_upperband': supertrend.final_upperband,
            'final_lowerband': supertrend.final_lowerband,
            'ADX': adx.adx
        }
    )

In [10]:
last_data = run_pipeline(make_pipeline(), last_date, last_date)
buy_data = last_data[last_data['close'] > last_data['final_upperband']]
buy_data

close  final_upperband  \
2024-10-25 00:00:00+00:00 Equity(16 [2354])   69.8        69.014286   
                          Equity(73 [4919])   94.1        89.441837   

                                             final_lowerband        ADX  
2024-10-25 00:00:00+00:00 Equity(16 [2354])        63.985714  79.029377  
                          Equity(73 [4919])        78.708163  79.987217

In [11]:
buy_list = []
for i in range(len(buy_data)):
    stock = buy_data.index.get_level_values(1)[i].symbol
    buy_list.append(stock)

buy_list

['2354', '4919']

In [12]:
sell_data = last_data[(last_data['close'] < last_data['final_lowerband']) & (last_data['ADX'] > 50)]
sell_data

close  final_upperband  \
2024-10-25 00:00:00+00:00 Equity(42 [2498])  45.85        52.910204   
                          Equity(93 [6770])  19.70        21.993878   

                                             final_lowerband        ADX  
2024-10-25 00:00:00+00:00 Equity(42 [2498])        47.089796  80.747982  
                          Equity(93 [6770])        19.831122  82.509597

In [13]:
sell_list = []
for i in range(len(sell_data)):
    stock = sell_data.index.get_level_values(1)[i].symbol
    sell_list.append(stock)

sell_list

['2498', '6770']

## Algorithmic Trading with SinoPac APIFor a detailed introduction to the Yongfeng API, please refer to: [Shioaji Documentation](https://sinotrade.github.io/)


In [ ]:
import shioaji as sj

api = sj.Shioaji(simulation=True) # Simulation mode
api.login(
    api_key = 'your api_key',     
    secret_key = 'your secret_key',
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.") # Fetch Contracts Callback
)

### Retrieving Current Holdings
The `list_positions` function in Sinopac API provides current holdings, including stock symbols, quantities, average prices, current prices, and profit/loss, etc.  

**Note**: If no positions are held, the function returns an empty DataFrame.

In [15]:
positions = api.list_positions(api.stock_account)
position_data = pd.DataFrame(s.__dict__ for s in positions)
position_data

""


### Fetching the list of holding stocks
Extracting the *code* column from the position data will provide the current list of held stocks, referred to as *hold_list*.

In [16]:
if position_data.empty:
    hold_list = []
else:
    hold_list = position_data['code'].to_list()
    
hold_list

[]

### Setting Up Order Callback
We define the order callback function `order_cb` and pass it to `set_order_callback`. Once an order is executed, it will return order details and trade execution messages.

In [17]:
def order_cb(stat, msg):
    print('my_order_callback')
    print(stat, msg)

api.set_order_callback(order_cb)

### Placing Orders
The place_order function is used to execute trades. It requires contract (stock information) and order (trade details). Key parameters include action (Buy/Sell), quantity, price type (MKT, MKP), order type (ROD, IOC, FOK), and order lot (Common, Odd), etc.  

The trading rules in this article are as follows:
 - If a stock in the buy_list is not in the hold_list, a MKT order will be placed to buy one share.
 - If a stock in the sell_list is in the hold_list, a MKT order will be placed to sell one share.

**Note**: Using price_type as MKP (market price with range) may result in the following error and prevent successful execution.  
```python
2024-09-24 09:52:51.492 | ERROR    | shioaji.shioaji:place_order:419 - {'status': {'status_code': 500}, 'response': {'detail': 'Internal Server Error'}}
```

In [18]:
for i in buy_list:
    print('processing %s' % i)
    
    if i not in hold_list:
        print('%s not in hold_list' % i)
        print('Ready to buy!')
        
        contract = api.Contracts.Stocks.TSE[i]

        order = api.Order(
            action = 'Buy',
            price = 0, # MKT, MKP will not use price parameter
            quantity = 1,
            price_type = 'MKT', # MKT or MKP
            order_type = 'IOC', # ROD, IOC, FOK
            order_lot = 'Common', # Common:整股, Fixing:定盤, Odd:盤後零股, IntradayOdd:盤中零股
            account = api.stock_account
        )

        trade = api.place_order(contract, order)
        trade

    else:
        print('%s in hold_list' % i)

    print('=' * 100)

processing 2354
2354 not in hold_list
Ready to buy!
processing 4919
4919 not in hold_list
Ready to buy!
my_order_callback
OrderState.StockOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '000401', 'seqno': '000401', 'ordno': '000106', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A95', 'account_id': '3369822', 'signed': True}, 'action': 'Buy', 'price': 0, 'quantity': 1, 'order_cond': 'Cash', 'order_lot': 'Common', 'custom_field': '', 'order_type': 'IOC', 'price_type': 'MKT'}, 'status': {'id': '000401', 'exchange_ts': 1730087221.832041, 'order_quantity': 1, 'modified_price': 0, 'cancel_quantity': 0, 'web_id': '137'}, 'contract': {'security_type': 'STK', 'exchange': 'TSE', 'code': '2354'}}
my_order_callback
OrderState.StockOrder {'operation': {'op_type': 'New', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '000402', 'seqno': '000402', 'ordno': '0000EE', 'account': {'account_type': 'S', 'person_id': '', 'broker_id': '9A95', 'accou

In [19]:
for i in sell_list:
    print('processing %s' % i)
    
    if i in hold_list:
        print('%s in hold_list' % i)
        print('Ready to sell!')
        
        contract = api.Contracts.Stocks.TSE[i]

        order = api.Order(
            action = 'Sell',
            price = 0, # MKT, MKP will not use price parameter
            quantity = 1,
            price_type = 'MKT', # MKT or MKP
            order_type = 'IOC', # ROD, IOC, FOK
            order_lot = 'Common', # Common:整股, Fixing:定盤, Odd:盤後零股, IntradayOdd:盤中零股
            account = api.stock_account
        )

        trade = api.place_order(contract, order)
        trade

    else:
        print('%s not in hold_list' % i)

    print('=' * 100)

processing 2498
2498 not in hold_list
processing 6770
6770 not in hold_list


In [20]:
positions = api.list_positions(api.stock_account)
position_data = pd.DataFrame(s.__dict__ for s in positions)
position_data

,id,code,direction,quantity,price,last_price,pnl,yd_quantity,cond,margin_purchase_amount,collateral,short_sale_margin,interest
0,0,2354,Action.Buy,1,72.9,72.9,-292.0,0,StockOrderCond.Cash,0,0,0,0
1,1,4919,Action.Buy,1,95.4,95.4,-381.0,0,StockOrderCond.Cash,0,0,0,0


### Viewing Realized P&L
The `list_profit_loss` function provides realized profit/loss data for a specified date range, including stock symbols, prices, quantities, and profit/loss details.

In [21]:
profitloss = api.list_profit_loss(api.stock_account,'2024-10-28','2024-10-28')
profitloss_data = pd.DataFrame(pnl.__dict__ for pnl in profitloss)
profitloss_data

,id,code,quantity,pnl,date,dseq,price,pr_ratio,cond,seqno
0,0,2330,1,112818.0,20241028,000158,1060.00,11.96379,StockOrderCond.Cash,00055D
1,1,2454,1,109777.0,20241028,000027,1320.00,9.11017,StockOrderCond.Cash,0000AB
2,2,2890,1,-1244.0,20241028,000030,23.35,-5.07745,StockOrderCond.Cash,0000D0
3,3,6152,1,-2915.0,20241028,000028,15.85,-15.58730,StockOrderCond.Cash,0000B2
4,4,2424,1,18687.0,20241028,00002C,80.60,30.33620,StockOrderCond.Cash,0000B0
5,5,6916,1,397.0,20241028,00002D,25.85,1.56548,StockOrderCond.Cash,0000B3
6,6,2488,1,-5294.0,20241028,000026,47.90,-9.98896,StockOrderCond.Cash,0000B1
7,7,6792,1,-2067.0,20241028,00019D,66.60,-3.02237,StockOrderCond.Cash,00066C
8,8,2466,1,-1540.0,20241028,00018D,34.90,-4.24325,StockOrderCond.Cash,00066B


### Logging Out
Due to API usage limits, it’s recommended to log out after completing your operations.

In [22]:
api.logout()

True